In [60]:
import pandas as pd
import numpy as np
import datetime
import lightgbm as lgb
import joblib
import graphviz 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'


#this section is for import and initial data cleanup

otomoto=pd.read_csv("output_otomoto_31052022.csv", header=None, dtype='unicode') #we drop header to remove duplicate header lines and again align it as header
otomoto.replace(np.nan, -1, inplace=True)

otomoto.drop_duplicates(inplace=True)

new_header = otomoto.iloc[0] #grab the first row for the header
otomoto = otomoto[1:] #take the data below the header row
otomoto.columns = new_header #set the header row as the df header
#otomoto.info()
#otomoto.to_csv('ototmoto_test.csv')



otomoto["model"]=otomoto["model"].astype("str")+" ("+otomoto["brand"].astype("str")+")"
otomoto=otomoto.astype("category")
otomoto["price"]=otomoto["price"].astype("str")
otomoto["price"]=otomoto["price"].str.replace(" ", "-1")
otomoto["price"]=otomoto["price"].str.replace(",", ".")
otomoto["price"]=otomoto["price"].astype("float")
otomoto["mileage"]=otomoto["mileage"].astype("str")
otomoto["mileage"]=otomoto["mileage"].str.replace("no_mileage", "-1")
otomoto["mileage"]=otomoto["mileage"].str.replace(" ", "-1")
otomoto["mileage"]=otomoto["mileage"].astype("int")
otomoto["year"]=otomoto["year"].astype("int")
otomoto["cylinder_capacity"]=otomoto["cylinder_capacity"].astype("str")
otomoto["cylinder_capacity"]=otomoto["cylinder_capacity"].str.replace("no_cylinder_capacity", "-1")
otomoto["cylinder_capacity"]=otomoto["cylinder_capacity"].str.replace(" ","-1")
otomoto["cylinder_capacity"]=otomoto["cylinder_capacity"].astype("int")
otomoto["damaged"]=otomoto["damaged"].replace(-1,"Nie")
otomoto["power"]=otomoto["power"].astype("str")
otomoto["power"]=otomoto["power"].str.replace("no_power", "-1")
otomoto["power"]=otomoto["power"].str.replace(" ","")
otomoto["power"]=otomoto["power"].astype("int")
otomoto["year"]=otomoto["year"].astype("int")
otomoto["current_year"]=datetime.datetime.now().year
otomoto["age"]=otomoto["current_year"]-otomoto["year"]
otomoto["version"]=otomoto["version"].replace({-1: "Brak"})
otomoto["vin"]=otomoto["vin"].astype("str")
otomoto["country"]=otomoto["country"].replace({-1:"Brak"})
otomoto["no_crash"]=otomoto["no_crash"].replace(-1,"Nie")
otomoto["model"]=otomoto["model"].astype("category")

otomoto.info()
otomoto


<class 'pandas.core.frame.DataFrame'>
Int64Index: 138054 entries, 1 to 138054
Data columns (total 44 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   ad_id                  138054 non-null  category
 1   brand                  138054 non-null  category
 2   car_type               138054 non-null  category
 3   city                   138054 non-null  category
 4   city_name              138054 non-null  category
 5   co2                    138054 non-null  category
 6   color                  138054 non-null  category
 7   colour_type            138054 non-null  category
 8   condition              138054 non-null  category
 9   country                138054 non-null  category
 10  cylinder_capacity      138054 non-null  int32   
 11  damaged                138054 non-null  category
 12  district               138054 non-null  category
 13  doors                  138054 non-null  category
 14  drive               

,ad_id,brand,car_type,city,city_name,co2,color,colour_type,condition,country,...,title,transmission,update_date,url,user_id,version,vin,year,current_year,age
1,6096996112,Abarth,Auta miejskie,lebork,-1,-1,Biały,metallic,Używane,Brak,...,abarth 595 stan salonowy polecam,Automatyczna,"20:41, 30 maja 2022",https://www.otomoto.pl/oferta/abarth-595-stan-...,no_user_id,Brak,ZFABF12H4MJF27047,2021,2022,1
2,6098071393,Abarth,Kompakt,paczkow,-1,-1,Inny kolor,metallic,Używane,Szwajcaria,...,abarth grande punto wersja ss 180 hp,Manualna,"13:12, 28 maja 2022",https://www.otomoto.pl/oferta/abarth-grande-pu...,no_user_id,Brak,xyzxyzxyzxyzxyzxz,2010,2022,12
3,6098072180,Abarth,Auta miejskie,krakow_136359,-1,-1,Biały,no_colour_type,Używane,Niemcy,...,abarth grande punto prywatny wlasciciel silnik...,Manualna,"14:01, 28 maja 2022",https://www.otomoto.pl/oferta/abarth-grande-pu...,no_user_id,Brak,ZFA19900000565386,2009,2022,13
4,6098075820,Abarth,Auta miejskie,legionowo,-1,-1,Biały,no_colour_type,Używane,Brak,...,abarth grande punto,Manualna,"20:46, 28 maja 2022",https://www.otomoto.pl/oferta/abarth-grande-pu...,no_user_id,Brak,ZFA19900000398058,2008,2022,14
5,6097804555,Abarth,Kompakt,zglobice,-1,-1,Czarny,metallic,Używane,Brak,...,abarth 595 abarth 595 turismo,Automatyczna,"12:50, 28 maja 2022",https://www.otomoto.pl/oferta/abarth-595-abart...,136148,Brak,ZFA3120000J865145,2017,2022,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138050,6097314709,Wartburg,Sedan,gdansk,-1,-1,Biały,no_colour_type,Używane,Niemcy,...,wartburg 353 wartburg 353,Manualna,"23:42, 24 maja 2022",https://www.otomoto.pl/oferta/wartburg-353-war...,no_user_id,Brak,23028959500000000,1988,2022,34
138051,6075023597,Wartburg,Sedan,koszalin,-1,-1,Niebieski,no_colour_type,Używane,Brak,...,wartburg 311 ddroska motoryzacja 15,Manualna,"14:07, 22 maja 2022",https://www.otomoto.pl/oferta/wartburg-311-ddr...,11680786,Brak,-1,1960,2022,62
138052,6092805586,Wartburg,Sedan,przasnysz,-1,-1,Inny kolor,matt,Używane,Polska,...,wartburg 353 sprzedam wartburga 353s 1987r ogl...,Manualna,"10:44, 23 maja 2022",https://www.otomoto.pl/oferta/wartburg-353-spr...,no_user_id,Brak,WVV122225376X123P,1987,2022,35
138053,6096899022,Wartburg,Sedan,janow-podlaski,-1,-1,Żółty,matt,Używane,Brak,...,wartburg 353 wartburg dwusuw zabytkowy prl,Manualna,"20:10, 22 maja 2022",https://www.otomoto.pl/oferta/wartburg-353-war...,no_user_id,Brak,10000000000000001,1982,2022,40


<class 'pandas.core.frame.DataFrame'>
Int64Index: 138054 entries, 1 to 138054
Data columns (total 44 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   ad_id                  138054 non-null  category
 1   brand                  138054 non-null  category
 2   car_type               138054 non-null  category
 3   city                   138054 non-null  category
 4   city_name              138054 non-null  category
 5   co2                    138054 non-null  category
 6   color                  138054 non-null  category
 7   colour_type            138054 non-null  category
 8   condition              138054 non-null  category
 9   country                138054 non-null  category
 10  cylinder_capacity      138054 non-null  int32   
 11  damaged                138054 non-null  category
 12  district               138054 non-null  category
 13  doors                  138054 non-null  category
 14  drive               

,ad_id,brand,car_type,city,city_name,co2,color,colour_type,condition,country,...,title,transmission,update_date,url,user_id,version,vin,year,current_year,age
1,6096996112,Abarth,Auta miejskie,lebork,-1,-1,Biały,metallic,Używane,Brak,...,abarth 595 stan salonowy polecam,Automatyczna,"20:41, 30 maja 2022",https://www.otomoto.pl/oferta/abarth-595-stan-...,no_user_id,Brak,ZFABF12H4MJF27047,2021,2022,1
2,6098071393,Abarth,Kompakt,paczkow,-1,-1,Inny kolor,metallic,Używane,Szwajcaria,...,abarth grande punto wersja ss 180 hp,Manualna,"13:12, 28 maja 2022",https://www.otomoto.pl/oferta/abarth-grande-pu...,no_user_id,Brak,xyzxyzxyzxyzxyzxz,2010,2022,12
3,6098072180,Abarth,Auta miejskie,krakow_136359,-1,-1,Biały,no_colour_type,Używane,Niemcy,...,abarth grande punto prywatny wlasciciel silnik...,Manualna,"14:01, 28 maja 2022",https://www.otomoto.pl/oferta/abarth-grande-pu...,no_user_id,Brak,ZFA19900000565386,2009,2022,13
4,6098075820,Abarth,Auta miejskie,legionowo,-1,-1,Biały,no_colour_type,Używane,Brak,...,abarth grande punto,Manualna,"20:46, 28 maja 2022",https://www.otomoto.pl/oferta/abarth-grande-pu...,no_user_id,Brak,ZFA19900000398058,2008,2022,14
5,6097804555,Abarth,Kompakt,zglobice,-1,-1,Czarny,metallic,Używane,Brak,...,abarth 595 abarth 595 turismo,Automatyczna,"12:50, 28 maja 2022",https://www.otomoto.pl/oferta/abarth-595-abart...,136148,Brak,ZFA3120000J865145,2017,2022,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138050,6097314709,Wartburg,Sedan,gdansk,-1,-1,Biały,no_colour_type,Używane,Niemcy,...,wartburg 353 wartburg 353,Manualna,"23:42, 24 maja 2022",https://www.otomoto.pl/oferta/wartburg-353-war...,no_user_id,Brak,23028959500000000,1988,2022,34
138051,6075023597,Wartburg,Sedan,koszalin,-1,-1,Niebieski,no_colour_type,Używane,Brak,...,wartburg 311 ddroska motoryzacja 15,Manualna,"14:07, 22 maja 2022",https://www.otomoto.pl/oferta/wartburg-311-ddr...,11680786,Brak,-1,1960,2022,62
138052,6092805586,Wartburg,Sedan,przasnysz,-1,-1,Inny kolor,matt,Używane,Polska,...,wartburg 353 sprzedam wartburga 353s 1987r ogl...,Manualna,"10:44, 23 maja 2022",https://www.otomoto.pl/oferta/wartburg-353-spr...,no_user_id,Brak,WVV122225376X123P,1987,2022,35
138053,6096899022,Wartburg,Sedan,janow-podlaski,-1,-1,Żółty,matt,Używane,Brak,...,wartburg 353 wartburg dwusuw zabytkowy prl,Manualna,"20:10, 22 maja 2022",https://www.otomoto.pl/oferta/wartburg-353-war...,no_user_id,Brak,10000000000000001,1982,2022,40


In [61]:
#new df based on otomoto

cars_dataset=otomoto[["price", "fuel", "power", "transmission", "mileage","year", "brand", "model", "no_crash", "damaged", "color", "country" ]].copy()
cars_dataset.sort_values(by=["year"], ascending=False, inplace=True)
cars_dataset
print(list(cars_dataset.year.unique()))
cars_dataset.info()
cars_dataset
#my_df=pd.DataFrame(brand)
#my_df.to_csv("brand.csv")

[2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950, 1949, 1948, 1947, 1946, 1939, 1938, 1937, 1936, 1935, 1934, 1932, 1931, 1929, 1928, 1926, 1924, 1915, 1901, -1]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 138054 entries, 5435 to 76933
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   price         138054 non-null  float64 
 1   fuel          138054 non-null  category
 2   power         138054 non-null  int32   
 3   transmission  138054 non-null  category
 4   mileage       138054 non-null  int32   
 5   year      

,price,fuel,power,transmission,mileage,year,brand,model,no_crash,damaged,color,country
5435,773700.0,Benzyna,600,Automatyczna,10,2023,Audi,RS Q8 (Audi),Tak,Nie,Niebieski,Polska
54360,133700.0,Hybryda,141,Automatyczna,10,2023,Kia,Niro (Kia),Tak,Nie,Niebieski,Polska
26537,182900.0,Benzyna,180,Automatyczna,5,2023,Citroën,Inny (Citroën),Tak,Nie,Fioletowy,Polska
54362,167900.0,Elektryczny,204,Automatyczna,10,2023,Kia,Niro (Kia),Tak,Nie,Biały,Polska
5514,775800.0,Benzyna,600,Automatyczna,10,2023,Audi,RS6 (Audi),Tak,Nie,Czerwony,Polska
...,...,...,...,...,...,...,...,...,...,...,...,...
24803,33900.0,Benzyna,20,Manualna,1,1926,Citroën,Inny (Citroën),Nie,Nie,Zielony,Brak
50094,119900.0,Benzyna,86,Manualna,3298,1924,Inny,Inny (Inny),Nie,Nie,Czarny,Brak
39236,149900.0,Benzyna,16,Manualna,5,1915,Ford,Inny (Ford),Tak,Nie,Czarny,Brak
50080,9000.0,Benzyna,7,Manualna,1880,1901,Inny,Inny (Inny),Nie,Nie,Czarny,Brak


In [73]:
#Train
# To define the input and output feature
#x = otomoto.drop(['price','url', 'number', 'year', 'current_year'],axis=1)
x = cars_dataset.drop(['price'],axis=1)
y = cars_dataset.price
# train and test split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=42)



model_lgbm = lgb.LGBMRegressor(bagging_freq=60, colsample_bytree=0.55, drop_rate=0.5, extra_trees=False, lambda_l1=0.6, lambda_l2=0.6, learning_rate=0.05,max_depth=8,n_estimators=364, num_leavers=95, subsample=0.65, random_state=42)
model_lgbm.fit(x_train,y_train, eval_set=[(x_test,y_test),(x_train,y_train)], verbose=20)
model_lgbm.booster_.save_model('lgbr_base.txt')
# save model
joblib.dump(model_lgbm, 'lgbr_model.pkl')

y_pred=model_lgbm.predict(x_test)


print('Training accuracy {:.4f}'.format(model_lgbm.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model_lgbm.score(x_test,y_test)))

ypred2=model_lgbm.predict(x) #predic values based on original data 
ypred2[0:10]  # showing first 5 predictions


fuel="Benzyna"
power=190
transmission="Automatyczna"
mileage=32000
year=2019
brand="Audi"
model="A5 (Audi)"
no_crash="Nie"
damaged="Nie"
color="Szary"
country="Austria"

df_pred=pd.DataFrame([[fuel,power,transmission,mileage, year, brand, model, no_crash,damaged, color, country ]], columns=['fuel','power','transmission','mileage', 'year', 'brand', 'model', 'no_crash','damaged', 'color', 'country'])
df_pred["fuel"]=df_pred["fuel"].astype("category")
df_pred["power"]=df_pred["power"].astype("int32" )
df_pred["transmission"]=df_pred["transmission"].astype("category")
df_pred["mileage"]=df_pred["mileage"].astype("int32")
df_pred["year"]=df_pred["year"].astype("int32")
df_pred["brand"]=df_pred["brand"].astype("category")
df_pred["model"]=df_pred["model"].astype("category")
df_pred["no_crash"]=df_pred["no_crash"].astype("category")
df_pred["damaged"]=df_pred["damaged"].astype("category")
df_pred["color"]=df_pred["color"].astype("category")
df_pred["country"]=df_pred["country"].astype("category")

#model = joblib.load('lgbr_model.pkl')
#model = lgb.load('lgbr_base.txt')
prediction = model_lgbm.predict(df_pred)
print(prediction)

C:\Users\pawel\AppData\Roaming\Python\Python39\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\pawel\AppData\Roaming\Python\Python39\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\pawel\AppData\Roaming\Python\Python39\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Unknown parameter: num_leavers
[LightGBM] [Warning] lambda_l1 is set=0.6, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6
[LightGBM] [Warning] lambda_l2 is set=0.6, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6
[LightGBM] [Warning] bagging_freq is set=60, subsample_freq=0 will be ignored. Current value: bagging_freq=60
[20]	training's l2: 3.00974e+09	valid_0's l2: 3.34898e+09
[40]	training's l2: 1.57343e+09	valid_0's l2: 1.85958e+09
[60]	training's l2: 1.16618e+09	valid_0's l2: 1.42067e+09
[80]	training's l2: 9.79333e+08	valid_0's l2: 1.18394e+09
[100]	training's l2: 8.94744e+08	valid_0's l2: 1.08026e+09
[120]	training's l2: 8.40184e+08	valid_0's l2: 1.02704e+09
[140]	training's l2: 8.00333e+08	valid_0's l2: 1.00817e+09
[160]	training's l2: 7.70948e+08	valid_0's l2: 9.97929e+08
[180]	training's l2: 7.48545e+08	valid_0's l2: 9.86451e+08
[200]	training's l2: 7.19658e+08	valid_0's l2: 9.54232e+08
[220]	training's l2: 6.9952e+08	valid_0's l2:

In [ ]:
from lightgbm import LGBMRegressor
import scipy
import numpy as np
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import cross_val_score
import sklearn
from sklearn import datasets
import pandas as pd



X=cars_dataset.drop(['price'],axis=1)
y=cars_dataset.price

n_estimators=scipy.stats.randint(50,400)
max_depth=scipy.stats.randint(2,10)
lambda_l1=np.arange(0,0.8,0.05)
lambda_l2=np.arange(0,0.8,0.05)
extra_trees=[True,False]
subsample=np.arange(0.3,1.0,0.05)
bagging_freq=scipy.stats.randint(1,100)
colsample_bytree=np.arange(0.3,1.0,0.05)
num_leaves=scipy.stats.randint(10,100)
boosting=['gbdt','dart']
drop_rate=np.arange(0.1,0.8,0.1)
skip_drop=np.arange(0,0.7,0.1)
learning_rate=[0.00001,0.0001,0.001,0.01,0.015,0.02,0.05,0.1,0.15,0.2]
params = {'n_estimators':n_estimators,'max_depth':max_depth, 'subsample':subsample,
          'colsample_bytree':colsample_bytree,
          'learning_rate':learning_rate,
          'num_leaves':num_leaves,
          'boosting':boosting,
          'extra_trees':extra_trees,
          'lambda_l1':lambda_l1,
          'lambda_l2':lambda_l2,
          'bagging_freq':bagging_freq,
          'drop_rate':drop_rate}


params = list(ParameterSampler(params, n_iter=500))
print(len(params))
print(params)

store=[]
for i in range(len(params)):
    print(i)
    p=params[i]
    model_lgbm=LGBMRegressor(**p,objective='rmse')

    score=cross_val_score(model,X,y,cv=10)
    
    store.append({'parameters':p,'r2':score})
    
    
    print(score)
    print(i)
    
store=pd.DataFrame(store)
store.to_csv('LGBM_parameters_search.csv')

In [ ]:
print('Training accuracy {:.4f}'.format(model_lgbm.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model_lgbm.score(x_test,y_test)))

In [ ]:
lgb.plot_importance(model_lgbm)

In [ ]:
lgb.plot_metric(model_lgbm)

In [ ]:
lgb.plot_tree(model_lgbm, figsize=(200,200))

In [ ]:
metrics.plot_confusion_matrix(model_lgbm,x_test,y_pred)
#metrics.plot_confusion_matrix(model,x_test,y_test,cmap='Blues_r')

In [ ]:
#confusion_matrix(y_test, y_pred)
model.score(x_test, y_test)

 

In [ ]:
print(metrics.classification_report(y_test,model_lgbm.predict(x_test)))

In [63]:
ypred2=model_lgbm.predict(x) #predic values based on original data 
ypred2[0:10]  # showing first 5 predictions
#cars_dataset["ypred2"]=ypred2
#cars_dataset["url"]=otomoto["url"]
#cars_dataset["diff"]=cars_dataset["ypred2"]-cars_dataset["price"]
#cars_dataset.to_csv("model_resulst.csv")

array([829606.99799546, 155069.55350276, 188265.08100411, 210511.40117005,
       746727.98162603, 229208.38068468, 219564.90733012,  95226.89555105,
        98540.78619833,  84979.91688339])

In [ ]:
x_test[0:10]
#y_test

In [ ]:
y_test[0:10]

In [ ]:
print(metrics.r2_score(y_test, ypred2))
#print(metrics.mean_squared_log_error(y_test, ypred2))


In [ ]:
params.to_csv('LGBM_parameters.csv')

['lgbr_model.pkl']

In [64]:

fuel="Benzyna"
power=190
mileage=32000
transmission="Automatyczna"
year=2019
brand="Audi"
model="A5 (Audi)"
no_crash="Nie"
damaged="Nie"
color="Szary"
country="Austria"

df_pred=pd.DataFrame([[fuel,power,mileage, transmission, year, brand, model, no_crash,damaged, color, country ]], columns=['fuel','power','mileage', 'transmission', 'year', 'brand', 'model', 'no_crash','damaged', 'color', 'country'])
df_pred["fuel"]=df_pred["fuel"].astype("category")
df_pred["power"]=df_pred["power"].astype("int32" )
df_pred["mileage"]=df_pred["mileage"].astype("int32")
df_pred["transmission"]=df_pred["transmission"].astype("category")
df_pred["year"]=df_pred["year"].astype("int32")
df_pred["brand"]=df_pred["brand"].astype("category")
df_pred["model"]=df_pred["model"].astype("category")
df_pred["no_crash"]=df_pred["no_crash"].astype("category")
df_pred["damaged"]=df_pred["damaged"].astype("category")
df_pred["color"]=df_pred["color"].astype("category")
df_pred["country"]=df_pred["country"].astype("category")

#model = joblib.load('lgbr_model.pkl')
#model = lgb.load('lgbr_base.txt')
prediction = model.predict(df_pred)
prediction

AttributeError: 'str' object has no attribute 'predict'

AttributeError: 'str' object has no attribute 'predict'